In [8]:
import psycopg2
import pandas as pd

# Database connection details
username = 'test'
password = 'test123'
hostname = '192.168.1.141'
port = '5432'
database_name = 'mydb'

# Establish connection
conn = psycopg2.connect(
    dbname=database_name,
    user=username,
    password=password,
    host=hostname,
    port=port
)

# SQL query

sql_query = "SELECT TO_TIMESTAMP(lasttradetime) AT TIME ZONE 'Asia/Kolkata' AS time,open, high, low, close,tradedqty as volume FROM whole_indices_futures WHERE instrumentidentifier = 'BANKNIFTY24APR24FUT'AND TO_TIMESTAMP(lasttradetime) AT TIME ZONE 'Asia/Kolkata' <= '2024-04-09 15:30:00' ORDER BY time;"
# Execute SQL query
cursor = conn.cursor()
cursor.execute(sql_query)


# Fetch all the results
rows = cursor.fetchall()

print(rows)
# Convert to DataFrame
df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])

# Close cursor and connection
cursor.close()
conn.close()

# Display the DataFrame
print(df)


[(datetime.datetime(2024, 3, 1, 9, 15), Decimal('46868.90'), Decimal('47152.10'), Decimal('46868.90'), Decimal('47131.80'), 3885), (datetime.datetime(2024, 3, 1, 9, 16), Decimal('47131.80'), Decimal('47160.30'), Decimal('47112.60'), Decimal('47114.10'), 3600), (datetime.datetime(2024, 3, 1, 9, 17), Decimal('47114.10'), Decimal('47131.00'), Decimal('47114.10'), Decimal('47120.00'), 1035), (datetime.datetime(2024, 3, 1, 9, 18), Decimal('47118.45'), Decimal('47150.00'), Decimal('47117.15'), Decimal('47117.65'), 1455), (datetime.datetime(2024, 3, 1, 9, 19), Decimal('47117.65'), Decimal('47133.90'), Decimal('47113.65'), Decimal('47125.20'), 1095), (datetime.datetime(2024, 3, 1, 9, 20), Decimal('47125.20'), Decimal('47135.00'), Decimal('47118.40'), Decimal('47130.00'), 705), (datetime.datetime(2024, 3, 1, 9, 21), Decimal('47130.00'), Decimal('47135.00'), Decimal('47118.30'), Decimal('47118.30'), 270), (datetime.datetime(2024, 3, 1, 9, 22), Decimal('47113.70'), Decimal('47188.45'), Decimal('4

In [9]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.layers import Dense

In [10]:
df.set_index('time', inplace=True)	
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)
lookback_window =3
features = 5
def create_sequences(data, lookback_window):
    X, y = [], []
    for i in range(len(data) - lookback_window - 1): 
        X.append(data[i:(i + lookback_window), :])
        y.append(data[i + lookback_window, 0:5]) 
    return np.array(X), np.array(y)
model = Sequential()
model.add(LSTM(units=200, return_sequences=True, input_shape=(lookback_window, features)))
model.add(LSTM(units=200))
model.add(Dense(200, activation=tf.nn.leaky_relu))
model.add(Dense(200, activation=tf.nn.leaky_relu))
model.add(Dense(units=5)) 
model.compile(loss='mean_squared_error', optimizer='adam')
X, y = create_sequences(df_scaled, lookback_window)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]




model.fit(X_train, y_train, epochs=50, batch_size=375, validation_data=(X_test, y_test))
y_pred = model.predict(X_test)
y_pred_original = scaler.inverse_transform(y_pred)
data = pd.DataFrame(y_pred_original, columns=['open', 'high', 'low' ,'close','tradedqty'])
data


KeyError: "None of ['lasttradetime'] are in the columns"

In [8]:
def generate_predictions(model, last_sequence, num_predictions):
    predictions = []
    current_sequence = last_sequence.reshape(1, lookback_window, features)  # Reshape to match model input shape
    for _ in range(num_predictions):
        next_prediction = model.predict(current_sequence)  # Predict the next data point
        predictions.append(next_prediction[0])  # Append the prediction to the list of predictions
        current_sequence = np.append(current_sequence[:, 1:, :], next_prediction.reshape(1, 1, features), axis=1)  # Update the current sequence
    return predictions

# Generate predictions for the next 60 data points
last_sequence = X_test[-1]  # Using the last sequence from the test data
num_predictions = 720
predicted_data = generate_predictions(model, last_sequence, num_predictions)

# Inverse transform the predictions to obtain the original scale
predicted_data_original = scaler.inverse_transform(predicted_data)

# Create a DataFrame with the predicted values and column names
predicted_df = pd.DataFrame(predicted_data_original, columns=['open', 'high', 'low', 'close', 'volume'])

# Print the predicted data
print(predicted_df)

    

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━

In [9]:
frequency = 'T'
start_time = '2024-04-10 15:31:00'
end_time = pd.to_datetime(start_time) + pd.DateOffset(minutes=len(predicted_df) - 1)
date_range = pd.date_range(start=start_time, end=end_time, freq=frequency)
predicted_df['lasttradetime'] = date_range
predicted_df.set_index('lasttradetime', inplace=True)
predicted_df.to_csv('stockwithdense.csv')


predicted_df

C:\Users\santh\AppData\Local\Temp\ipykernel_11508\3572586800.py:4: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  date_range = pd.date_range(start=start_time, end=end_time, freq=frequency)


,open,high,low,close,volume
lasttradetime,,,,,
2024-04-10 15:31:00,52.979910,54.829779,52.729530,53.599595,4.953524e+06
2024-04-10 15:32:00,53.092735,54.860500,52.759022,53.743898,4.806042e+06
2024-04-10 15:33:00,53.257701,54.926429,52.805051,53.874032,4.612930e+06
2024-04-10 15:34:00,53.369474,54.893196,52.815839,53.869351,4.492070e+06
2024-04-10 15:35:00,53.476628,54.852408,52.865719,53.865187,4.365839e+06
...,...,...,...,...,...
2024-04-11 03:26:00,41.199655,41.447606,41.152396,41.201652,2.228597e+06
2024-04-11 03:27:00,41.199655,41.447606,41.152399,41.201652,2.228597e+06
2024-04-11 03:28:00,41.199655,41.447606,41.152396,41.201655,2.228597e+06
